In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:

import os
!pip install Keras-Preprocessing
import tensorflow as tf
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("ggplot")
%matplotlib inline

from tqdm import tqdm_notebook, tnrange
from itertools import chain
from skimage.io import imread, imshow, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from sklearn.model_selection import train_test_split
import imageio

from keras.models import Model, load_model
from keras.layers import Input, BatchNormalization, Activation, Dense, Dropout, Resizing
from keras.layers import Lambda, RepeatVector, Reshape
from keras.layers import Conv2D, Conv2DTranspose
from keras.layers import MaxPooling2D, GlobalMaxPool2D, AveragePooling2D
from keras.regularizers import l2
#from keras.layers.merge import concatenate, add
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers import Adam
from keras_preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import *
from keras.losses import *
from keras import backend as keras
# from keras.applications.vgg16 import VGG16, preprocess_input
# from keras.applications.densenet import DenseNet201, DenseNet121
# from keras.applications import EfficientNetV2S, EfficientNetV2B1
from keras.utils import plot_model
# from keras.utils import plot_model
from keras.preprocessing.image import ImageDataGenerator
import cv2
import skimage.io as io
# import skimage.transform as trans
import glob
from PIL import Image
import skimage
from keras.initializers import Constant

from skimage.morphology import disk
from sklearn.metrics import confusion_matrix
from skimage.measure import label, regionprops
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import jaccard_score

from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras import layers, models
import zipfile
import random
import sys

import skimage.io                           #Used for imshow function
import skimage.transform                    #Used for resize function
from skimage.morphology import label        #Used for Run-Length-Encoding RLE to create final submission
import matplotlib.pyplot as plt
from keras.layers import Concatenate, add

import warnings
warnings.filterwarnings("ignore")
random.seed(42)
from keras.losses import binary_crossentropy
from sklearn.metrics import precision_recall_curve

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 928.1 kB/s eta 0:00:00


In [3]:
from keras import backend as K
from keras.losses import binary_crossentropy
def dice_coef(y_true, y_pred,smooth = 10):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

def Jaccard_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (intersection ) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection)

def Jaccard_coef_loss(y_true, y_pred):
    return (1-Jaccard_coef(y_true, y_pred))

# def bcc_Jaccard_coef_loss(y_true, y_pred):
#     return (binary_crossentropy(y_true, y_pred)+(1-Jaccard_coef(y_true, y_pred)))

#Metrices
from keras import backend as K
import tensorflow as tf
import keras

def iou(y_true, y_pred, threshold=0.5):
    y_pred = threshold_binarize(y_pred, threshold)
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (intersection) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection)

def dice_coef(y_true, y_pred, threshold=0.5):
    y_pred = threshold_binarize(y_pred, threshold)
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection) / (K.sum(y_true_f) + K.sum(y_pred_f))

def sensitivity(y_train, results):
    true_positives = K.sum(K.round(K.clip(y_train * results, 0, 1)))
    actual_positives = K.sum(K.round(K.clip(y_train, 0, 1)))
    sensitivity = true_positives / (actual_positives + K.epsilon())
    return sensitivity

def specificity(y_true, y_pred):
    true_negatives = K.sum(K.round(K.clip((1-y_true) * (1-y_pred), 0, 1)))
    possible_negatives = K.sum(K.round(K.clip(1-y_true, 0, 1)))
    return true_negatives / (possible_negatives + K.epsilon())

def threshold_binarize(x, threshold=0.5):
    ge = tf.greater_equal(x, tf.constant(threshold))
    y = tf.where(ge, x=tf.ones_like(x), y=tf.zeros_like(x))
    return y

def DiceLoss(y_true, y_pred, smooth=1e-6):
    y_true, y_pred = tf.cast(y_true, dtype=tf.float32), tf.cast(y_pred, tf.float32)
    nominator = 2 * tf.reduce_sum(tf.multiply(y_pred, y_true)) + smooth
    denominator = tf.reduce_sum(y_pred ** 2) + tf.reduce_sum(y_true ** 2) + smooth
    result = 1 - tf.divide(nominator, denominator)
    return result


from keras import backend as K
import numpy as np
import tensorflow as tf
from scipy.ndimage import distance_transform_edt as distance


def calc_dist_map(seg):
    res = np.zeros_like(seg)
    posmask = seg.astype(np.bool)

    if posmask.any():
        negmask = ~posmask
        res = distance(negmask) * negmask - (distance(posmask) - 1) * posmask

    return res


def calc_dist_map_batch(y_true):
    y_true_numpy = y_true.numpy()
    return np.array([calc_dist_map(y)
                     for y in y_true_numpy]).reshape(y_true.shape).astype(np.float32)


def surface_loss_keras(y_true, y_pred):
    y_true_dist_map = tf.py_function(func=calc_dist_map_batch,
                                     inp=[y_true],
                                     Tout=tf.float32)
    multipled = y_pred * y_true_dist_map
    return K.mean(multipled)


def bcc_Jaccard_coef_loss(y_true, y_pred):
    return (DiceLoss(y_true, y_pred, smooth=1e-6)+(1-iou(y_true, y_pred, threshold=0.5)))
    #return (DiceLoss(y_true, y_pred, smooth=1e-6)+(1-iou(y_true, y_pred, threshold=0.5))+ (0.01*surface_loss_keras(y_true, y_pred)))

**Transfuse Net**


In [19]:
from keras.layers import GlobalAveragePooling2D, GlobalMaxPooling2D, Reshape, Dense, multiply, Permute, Concatenate
from keras import backend as K

def cbam_block(cbam_feature, ratio=8):
    """Contains the implementation of Convolutional Block Attention Module(CBAM) block.
    As described in https://arxiv.org/abs/1807.06521.
    """

    cbam_feature = channel_attention(cbam_feature, ratio)
    cbam_feature = spatial_attention(cbam_feature)
    return cbam_feature

def channel_attention(input_feature, ratio=8):
    channel_axis = 1 if K.image_data_format() == "channels_first" else -1
    channel = input_feature.shape[channel_axis]

    shared_layer_one = Dense(channel//ratio,
                             activation='relu',
                             kernel_initializer='he_normal',

                             use_bias=True,
                             bias_initializer='zeros')
    shared_layer_two = Dense(channel,
                             kernel_initializer='he_normal',
                             use_bias=True,
                             bias_initializer='zeros')

    avg_pool = GlobalAveragePooling2D()(input_feature)
    avg_pool = Reshape((1,1,channel))(avg_pool)
    assert avg_pool.shape[1:] == (1,1,channel)
    avg_pool = shared_layer_one(avg_pool)
    assert avg_pool.shape[1:] == (1,1,channel//ratio)
    avg_pool = shared_layer_two(avg_pool)
    assert avg_pool.shape[1:] == (1,1,channel)

    max_pool = GlobalMaxPooling2D()(input_feature)
    max_pool = Reshape((1,1,channel))(max_pool)
    assert max_pool.shape[1:] == (1,1,channel)
    max_pool = shared_layer_one(max_pool)
    assert max_pool.shape[1:] == (1,1,channel//ratio)
    max_pool = shared_layer_two(max_pool)
    assert max_pool.shape[1:] == (1,1,channel)

    cbam_feature = Add()([avg_pool,max_pool])
    cbam_feature = Activation('sigmoid')(cbam_feature)

    if K.image_data_format() == "channels_first":
        cbam_feature = Permute((3, 1, 2))(cbam_feature)

    return multiply([input_feature, cbam_feature])

def spatial_attention(input_feature):
    kernel_size = 7

    if K.image_data_format() == "channels_first":
        channel = input_feature.shape[1]
        cbam_feature = Permute((2,3,1))(input_feature)
    else:
        channel = input_feature.shape[-1]
        cbam_feature = input_feature

    avg_pool = Lambda(lambda x: K.mean(x, axis=3, keepdims=True))(cbam_feature)
    assert avg_pool.shape[-1] == 1
    max_pool = Lambda(lambda x: K.max(x, axis=3, keepdims=True))(cbam_feature)
    assert max_pool.shape[-1] == 1
    concat = Concatenate(axis=3)([avg_pool, max_pool])
    assert concat.shape[-1] == 2
    cbam_feature = Conv2D(filters = 1,
                    kernel_size=kernel_size,
                    strides=1,
                    padding='same',
                    activation='sigmoid',
                    kernel_initializer='he_normal',
                    use_bias=False)(concat)
    assert cbam_feature.shape[-1] == 1

    if K.image_data_format() == "channels_first":
        cbam_feature = Permute((3, 1, 2))(cbam_feature)

    return multiply([input_feature, cbam_feature])

# def get_positional_encoding(seq_len, d_model):
#     positions = tf.range(seq_len, dtype=tf.float32)[:, tf.newaxis]
#     div_term = tf.exp(tf.range(0, d_model, 2, dtype=tf.float32) * -(tf.math.log(10000.0) / d_model))
#     sine = tf.sin(positions * div_term)
#     cosine = tf.cos(positions * div_term)
#     pos_enc = tf.concat([sine, cosine], axis=-1)
#     pos_enc = tf.reshape(pos_enc, (seq_len, d_model))
#     return pos_enc



def create_transfuse_net(input_shape):
    # Define the input layer
    inputs = layers.Input(shape=input_shape)

    # Encoder - Convolutional Blocks
    conv_block1 = layers.Conv2D(8, kernel_size=(3, 3), padding='same', activation='relu')(inputs)
    conv_block1 = layers.MaxPooling2D(pool_size=(2, 2))(conv_block1)
    conv_block1 = layers.BatchNormalization()(conv_block1)

    conv_block2 = layers.Conv2D(16, kernel_size=(3, 3), padding='same', activation='relu')(conv_block1)
    conv_block2 = layers.MaxPooling2D(pool_size=(2, 2))(conv_block2)
    conv_block2 = layers.BatchNormalization()(conv_block2)

    conv_block3 = layers.Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu')(conv_block2)
    conv_block3 = layers.MaxPooling2D(pool_size=(2, 2))(conv_block3)
    conv_block3 = layers.BatchNormalization()(conv_block3)

    # Encoder - Transformer Block
    transformer_block = layers.Reshape((-1, conv_block3.shape[3]))(conv_block3)


    # # Add positional encoding
    # seq_len = transformer_block.shape[1]
    # d_model = transformer_block.shape[2]
    # pos_enc = get_positional_encoding(seq_len, d_model)
    # transformer_block = tf.add(transformer_block, pos_enc)



    transformer_block = layers.MultiHeadAttention(num_heads=4, key_dim=32)(transformer_block, transformer_block)
    transformer_block = layers.GlobalAveragePooling1D()(transformer_block)
    transformer_block = layers.Reshape((1, 1, transformer_block.shape[1]))(transformer_block)

    # Tile the transformer block output to match the spatial dimensions of the convolutional block
    transformer_block = layers.Lambda(lambda x: tf.tile(x, [1, conv_block3.shape[1], conv_block3.shape[2], 1]))(transformer_block)

    # Concatenate the Encoder Convolutional and Transformer blocks

    att1 = cbam_block(transformer_block)
    fused_features = layers.Concatenate()([conv_block3, att1])

    # Decoder - Upsampling Blocks
    decoder_block1 = layers.Conv2DTranspose(32, kernel_size=(3, 3), strides=(2, 2), padding='same', activation='relu')(fused_features)
    att2=cbam_block(decoder_block1)
    decoder_block1 = layers.Concatenate()([att2, conv_block2])
    decoder_block1 = layers.Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu')(decoder_block1)

    decoder_block2 = layers.Conv2DTranspose(16, kernel_size=(3, 3), strides=(2, 2), padding='same', activation='relu')(decoder_block1)

    att3=cbam_block(decoder_block2)
    decoder_block2 = layers.Concatenate()([att3, conv_block1])
    decoder_block2 = layers.Conv2D(16, kernel_size=(3, 3), padding='same', activation='relu')(decoder_block2)


    decoder_block3 = layers.Conv2DTranspose(8, kernel_size=(3, 3), strides=(2, 2), padding='same', activation='relu')(decoder_block2)
    decoder_block3 = layers.Conv2D(8, kernel_size=(3, 3), padding='same', activation='relu')(decoder_block3)
    # Decoder - Output Block
    output_BV = layers.Conv2D(1, kernel_size=(1, 1), activation='sigmoid',name = 'final_output1')(decoder_block3)
    output_OD = layers.Conv2D(1, kernel_size=(1, 1), activation='sigmoid', name = 'final_output2')(decoder_block3)

    # Create the model
    model = models.Model(inputs=inputs, outputs=[output_BV, output_OD])

    return model

# Define the input shape for binary semantic segmentation
input_shape = (512,512,3)  # Example input shape for RGB images

# Create the TransFuse Encoder-Decoder model with 3 MaxPooling layers for binary semantic segmentation
model = create_transfuse_net(input_shape)


In [5]:
# def create_transfuse_net(input_shape):
#     # Define the input layer
#     inputs = layers.Input(shape=input_shape)

#     # Encoder - Convolutional Blocks
#     conv_block1 = layers.Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu')(inputs)
#     conv_block1 = layers.MaxPooling2D(pool_size=(2, 2))(conv_block1)
#     conv_block1 = layers.BatchNormalization()(conv_block1)

#     conv_block2 = layers.Conv2D(46, kernel_size=(3, 3), padding='same', activation='relu')(conv_block1)
#     conv_block2 = layers.MaxPooling2D(pool_size=(2, 2))(conv_block2)
#     conv_block2 = layers.BatchNormalization()(conv_block2)

#     conv_block3 = layers.Conv2D(128, kernel_size=(3, 3), padding='same', activation='relu')(conv_block2)
#     conv_block3 = layers.MaxPooling2D(pool_size=(2, 2))(conv_block3)
#     conv_block3 = layers.BatchNormalization()(conv_block3)

#     # Encoder - Transformer Block
#     transformer_block = layers.Reshape((-1, conv_block3.shape[3]))(conv_block3)
#     transformer_block = layers.MultiHeadAttention(num_heads=4, key_dim=32)(transformer_block, transformer_block)
#     transformer_block = layers.GlobalAveragePooling1D()(transformer_block)
#     transformer_block = layers.Reshape((1, 1, transformer_block.shape[1]))(transformer_block)

#     # Tile the transformer block output to match the spatial dimensions of the convolutional block
#     transformer_block = layers.Lambda(lambda x: tf.tile(x, [1, conv_block3.shape[1], conv_block3.shape[2], 1]))(transformer_block)

#     # Concatenate the Encoder Convolutional and Transformer blocks
#     fused_features = layers.Concatenate()([conv_block3, transformer_block])

#     # Decoder - Upsampling Blocks
#     decoder_block1 = layers.Conv2DTranspose(128, kernel_size=(3, 3), strides=(2, 2), padding='same', activation='relu')(fused_features)
#     decoder_block1 = layers.Concatenate()([decoder_block1, conv_block2])
#     decoder_block1 = layers.Conv2D(128, kernel_size=(3, 3), padding='same', activation='relu')(decoder_block1)

#     decoder_block2 = layers.Conv2DTranspose(64, kernel_size=(3, 3), strides=(2, 2), padding='same', activation='relu')(decoder_block1)
#     decoder_block2 = layers.Concatenate()([decoder_block2, conv_block1])
#     decoder_block2 = layers.Conv2D(64, kernel_size=(3, 3), padding='same', activation='relu')(decoder_block2)


#     decoder_block3 = layers.Conv2DTranspose(32, kernel_size=(3, 3), strides=(2, 2), padding='same', activation='relu')(decoder_block2)
#     decoder_block3 = layers.Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu')(decoder_block3)
#     # Decoder - Output Block
#     output_BV = layers.Conv2D(1, kernel_size=(1, 1), activation='sigmoid',name = 'final_output1')(decoder_block3)
#     output_OD = layers.Conv2D(1, kernel_size=(1, 1), activation='sigmoid', name = 'final_output2')(decoder_block3)

#     # Create the model
#     model = models.Model(inputs=inputs, outputs=[output_BV, output_OD])

#     return model

# # Define the input shape for binary semantic segmentation
# input_shape = (256,256, 3)  # Example input shape for RGB images

# # Create the TransFuse Encoder-Decoder model with 3 MaxPooling layers for binary semantic segmentation
# model = create_transfuse_net(input_shape)


**UNet++**

In [6]:
# SUPERVISION = True
# num_classes = 3
# act = "relu"
# from keras import regularizers
# from keras.layers import Concatenate
# def conv_block(inputs, filters, pool=True, drop=False):
#     x = Conv2D(filters, 3, padding="same")(inputs) #kernel_regularizer=regularizers.l2(0.0001)
#     #x = BatchNormalization()(x)
#     x = Activation("relu")(x)

#     x = Conv2D(filters, 3, padding="same")(x)
#     #x = BatchNormalization()(x)
#     x = Activation("relu")(x)

#     if pool == True and drop == True:
#         p = Dropout(0.5)(x)
#         p = MaxPool2D((2, 2))(p)
#         return x, p
#     elif pool == True and drop == False:
#         p = MaxPool2D((2, 2))(x)
#         return x, p
#     else:
#         return x

# def standard_unit(input_tensor, stage, nb_filter, kernel_size=3):

#     x = Conv2D(nb_filter, (kernel_size, kernel_size), activation=act, name='conv'+stage+'_1', kernel_initializer = 'he_normal', padding='same', kernel_regularizer=regularizers.l2(1e-4))(input_tensor)
#     #x = Dropout(dropout_rate, name='dp'+stage+'_1')(x)
#     x = Conv2D(nb_filter, (kernel_size, kernel_size), activation=act, name='conv'+stage+'_2', kernel_initializer = 'he_normal', padding='same', kernel_regularizer=regularizers.l2(1e-4))(x)
#     #x = Dropout(dropout_rate, name='dp'+stage+'_2')(x)

#     return x


# def UNetPlusPlus(img_rows, img_cols, color_type=3, num_class=1):

#     nb_filter = [16,32,64,128,256]

#     # Handle Dimension Ordering for different backends
#     global bn_axis
#     #if K.image_dim_ordering() == 'tf':
#     bn_axis = 3
#     img_input = Input(shape=(img_rows, img_cols, color_type), name='main_input')
#     #else:
#     #  bn_axis = 1
#     #  img_input = Input(shape=(color_type, img_rows, img_cols), name='main_input')

#     conv1_1 = standard_unit(img_input, stage='11', nb_filter=nb_filter[0])
#     pool1 = MaxPooling2D((2, 2), strides=(2, 2), name='pool1')(conv1_1)

#     conv2_1 = standard_unit(pool1, stage='21', nb_filter=nb_filter[1])
#     pool2 = MaxPooling2D((2, 2), strides=(2, 2), name='pool2')(conv2_1)

#     up1_2 = Conv2DTranspose(nb_filter[0], (2, 2), strides=(2, 2), name='up12', padding='same')(conv2_1)
#     conv1_2 = concatenate([up1_2, conv1_1], name='merge12', axis=bn_axis)
#     conv1_2 = standard_unit(conv1_2, stage='12', nb_filter=nb_filter[0])

#     conv3_1 = standard_unit(pool2, stage='31', nb_filter=nb_filter[2])
#     pool3 = MaxPooling2D((2, 2), strides=(2, 2), name='pool3')(conv3_1)

#     up2_2 = Conv2DTranspose(nb_filter[1], (2, 2), strides=(2, 2), name='up22', padding='same')(conv3_1)
#     conv2_2 = concatenate([up2_2, conv2_1], name='merge22', axis=bn_axis)
#     conv2_2 = standard_unit(conv2_2, stage='22', nb_filter=nb_filter[1])

#     up1_3 = Conv2DTranspose(nb_filter[0], (2, 2), strides=(2, 2), name='up13', padding='same')(conv2_2)
#     conv1_3 = concatenate([up1_3, conv1_1, conv1_2], name='merge13', axis=bn_axis)
#     conv1_3 = standard_unit(conv1_3, stage='13', nb_filter=nb_filter[0])

#     conv4_1 = standard_unit(pool3, stage='41', nb_filter=nb_filter[3])
#     pool4 = MaxPooling2D((2, 2), strides=(2, 2), name='pool4')(conv4_1)

#     up3_2 = Conv2DTranspose(nb_filter[2], (2, 2), strides=(2, 2), name='up32', padding='same')(conv4_1)
#     conv3_2 = concatenate([up3_2, conv3_1], name='merge32', axis=bn_axis)
#     conv3_2 = standard_unit(conv3_2, stage='32', nb_filter=nb_filter[2])

#     up2_3 = Conv2DTranspose(nb_filter[1], (2, 2), strides=(2, 2), name='up23', padding='same')(conv3_2)
#     conv2_3 = concatenate([up2_3, conv2_1, conv2_2], name='merge23', axis=bn_axis)
#     conv2_3 = standard_unit(conv2_3, stage='23', nb_filter=nb_filter[1])

#     up1_4 = Conv2DTranspose(nb_filter[0], (2, 2), strides=(2, 2), name='up14', padding='same')(conv2_3)
#     conv1_4 = concatenate([up1_4, conv1_1, conv1_2, conv1_3], name='merge14', axis=bn_axis)
#     conv1_4 = standard_unit(conv1_4, stage='14', nb_filter=nb_filter[0])

#     conv5_1 = standard_unit(pool4, stage='51', nb_filter=nb_filter[4])

#     up4_2 = Conv2DTranspose(nb_filter[3], (2, 2), strides=(2, 2), name='up42', padding='same')(conv5_1)
#     conv4_2 = concatenate([up4_2, conv4_1], name='merge42', axis=bn_axis)
#     conv4_2 = standard_unit(conv4_2, stage='42', nb_filter=nb_filter[3])

#     up3_3 = Conv2DTranspose(nb_filter[2], (2, 2), strides=(2, 2), name='up33', padding='same')(conv4_2)
#     conv3_3 = concatenate([up3_3, conv3_1, conv3_2], name='merge33', axis=bn_axis)
#     conv3_3 = standard_unit(conv3_3, stage='33', nb_filter=nb_filter[2])

#     up2_4 = Conv2DTranspose(nb_filter[1], (2, 2), strides=(2, 2), name='up24', padding='same')(conv3_3)
#     conv2_4 = concatenate([up2_4, conv2_1, conv2_2, conv2_3], name='merge24', axis=bn_axis)
#     conv2_4 = standard_unit(conv2_4, stage='24', nb_filter=nb_filter[1])

#     up1_5 = Conv2DTranspose(nb_filter[0], (2, 2), strides=(2, 2), name='up15', padding='same')(conv2_4)
#     conv1_5 = concatenate([up1_5, conv1_1, conv1_2, conv1_3, conv1_4], name='merge15', axis=bn_axis)
#     conv1_5 = standard_unit(conv1_5, stage='15', nb_filter=nb_filter[0])

#     # nestnet_output_1 = Conv2D(num_class, (1, 1), activation='sigmoid', name='output_1', kernel_initializer = 'he_normal', padding='same',kernel_regularizer=regularizers.l2(1e-4))(conv1_2)
#     # nestnet_output_2 = Conv2D(num_class, (1, 1), activation='sigmoid', name='output_2', kernel_initializer = 'he_normal', padding='same', kernel_regularizer=regularizers.l2(1e-4))(conv1_3)
#     nestnet_output_3 = Conv2D(1, (1, 1), activation='sigmoid', name='output_3', padding='same')(conv1_5)
#     nestnet_output_4 = Conv2D(1, (1, 1), activation='sigmoid', name='output_4',padding='same')(conv1_5)

#     model = Model(img_input, [nestnet_output_3, nestnet_output_4])
#     return model

# model = UNetPlusPlus(512,512,3, num_classes)
# #  loss = bcct_Jaccard_coef_loss


# model.summary()

**Attention UNet**

In [7]:
# # Attention module block

# def attention_block2d(x, g, inter_channel, i, data_format='channels_last'):

#     theta_x = Conv2D(inter_channel, [2, 2], strides=[2, 2], data_format=data_format, kernel_regularizer=regularizers.l2(1e-5))(x)

#     phi_g = Conv2D(inter_channel, [1, 1], strides=[1, 1],dilation_rate=i, data_format=data_format, kernel_regularizer=regularizers.l2(1e-5))(g)

#     f = LeakyReLU(alpha=0.2)(Add()([theta_x, phi_g]))

#     psi_f = Conv2D(1, [1, 1], strides=[1, 1],dilation_rate=i, data_format=data_format, kernel_regularizer=regularizers.l2(1e-5))(f)

#     sigm_psi_f = Activation(activation='sigmoid')(psi_f)

#     rate = UpSampling2D(size=[2, 2])(sigm_psi_f)

#     att_x = multiply([x, rate])

#     return att_x

# def res_block(x, nb_filters, strides, i):

#     res_path = BatchNormalization()(x)
#     res_path = LeakyReLU(alpha=0.2)(res_path)
#     pool = MaxPooling2D(pool_size=(2, 2))(res_path)
#     res_path = Conv2D(filters=nb_filters[0], kernel_size=(3, 3), padding='same',dilation_rate=i, strides=strides[1], kernel_regularizer=regularizers.l2(1e-5))(pool)
#     res_path = BatchNormalization()(res_path)
#     res_path = LeakyReLU(alpha=0.2)(res_path)
#     res_path = Conv2D(filters=nb_filters[1], kernel_size=(3, 3), padding='same', dilation_rate=i,strides=strides[1], kernel_regularizer=regularizers.l2(1e-5))(res_path)

#     shortcut = Conv2D(nb_filters[1], kernel_size=(1, 1),dilation_rate=i, strides=strides[1], kernel_regularizer=regularizers.l2(1e-5))(pool)
#     shortcut = BatchNormalization()(shortcut)

#     res_path = Add()([shortcut, res_path])
#     return res_path

# def encoder(x):
#     to_decoder = []

#     main_path = Conv2D(filters=64, kernel_size=(3, 3), padding='same',dilation_rate=(1,1), strides=(1, 1), kernel_regularizer=regularizers.l2(1e-5))(x)
#     main_path = BatchNormalization()(main_path)
#     main_path = LeakyReLU(alpha=0.2)(main_path)

#     main_path = Conv2D(filters=64, kernel_size=(3, 3), padding='same',dilation_rate=(1,1),strides=(1, 1), kernel_regularizer=regularizers.l2(1e-5))(main_path)

#     shortcut = Conv2D(filters=64, kernel_size=(1, 1), strides=(1, 1),dilation_rate=(1,1), kernel_regularizer=regularizers.l2(1e-5))(x)
#     shortcut = BatchNormalization()(shortcut)

#     main_path = Add()([shortcut, main_path])

#     # first branching to decoder
#     to_decoder.append(main_path)

#     main_path = res_block(main_path, [64, 64], [(2, 2), (1, 1)],(1,1))
#     to_decoder.append(main_path)

#     main_path = res_block(main_path, [128,128], [(2, 2), (1, 1)],(2,2))
#     to_decoder.append(main_path)

#     main_path = res_block(main_path, [256, 256], [(2, 2), (1, 1)],(4,4))
#     to_decoder.append(main_path)

#     return to_decoder

# def res_block_decoder(x, nb_filters, strides,i):

#     res_path = BatchNormalization()(x)
#     res_path = LeakyReLU(alpha=0.2)(res_path)
#     res_path = Conv2D(filters=nb_filters[0], kernel_size=(3, 3), padding='same', dilation_rate=i, strides=strides[1], kernel_regularizer=regularizers.l2(1e-5))(res_path)
#     res_path = BatchNormalization()(res_path)
#     res_path = LeakyReLU(alpha=0.2)(res_path)
#     res_path = Conv2D(filters=nb_filters[1], kernel_size=(3, 3), padding='same', dilation_rate=i, strides=strides[1], kernel_regularizer=regularizers.l2(1e-5))(res_path)

#     shortcut = Conv2D(nb_filters[1], kernel_size=(1, 1), strides=strides[1], dilation_rate=i, kernel_regularizer=regularizers.l2(1e-5))(x)
#     shortcut = BatchNormalization()(shortcut)

#     res_path = Add()([shortcut, res_path])

#     return res_path
# def decoder(x, from_encoder):

#     # Layer 4
#     attention_path1 = attention_block2d(from_encoder[3], x, 256, (1,1), data_format='channels_last')

#     main_path1 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same',dilation_rate=(1,1),kernel_regularizer=regularizers.l2(1e-5))(x)
#     main_path1 = Concatenate()([main_path1,attention_path1])
#     main_path1 = res_block_decoder(main_path1, [256,256], [(1, 1), (1, 1)],(4,4))

#     hc1 = three_times_sample(main_path1)
#     out_1 = Conv2D(filters=4, kernel_size=(1, 1), activation='softmax', name='out_1')(hc1)


#     # Layer 3
#     attention_path2 = attention_block2d(from_encoder[2],main_path1 ,128,(1,1),data_format='channels_last')

#     main_path2 = Conv2DTranspose(128, (2, 2), strides=(2, 2), dilation_rate=(1,1), padding='same', kernel_regularizer=regularizers.l2(1e-5))(main_path1)
#     main_path2 = Concatenate()([main_path2,attention_path2])
#     main_path2 = res_block_decoder(main_path2, [128, 128], [(1, 1), (1, 1)],(2,2))

#     hc2 = two_times_sample(main_path2)
#     out_2 = Conv2D(filters=4, kernel_size=(1, 1), activation='softmax', name='out_2')(hc2)


#     # Layer 2
#     attention_path3 = attention_block2d(from_encoder[1],main_path2,64,(1,1),data_format='channels_last')

#     main_path3 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same',dilation_rate=(1,1), kernel_regularizer=regularizers.l2(1e-5)) (main_path2)
#     main_path3 = Concatenate()([main_path3,attention_path3])
#     main_path3 = res_block_decoder(main_path3, [64,64], [(1, 1), (1, 1)],(1,1))

#     hc3 = one_time_sample(main_path3)
#     out_3 = Conv2D(filters=4, kernel_size=(1, 1), activation='softmax', name='out_3')(hc3)


#     # Layer 1
#     attention_path4 = attention_block2d(from_encoder[0],main_path3,32,(1,1),data_format='channels_last')

#     main_path4 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same',dilation_rate=(1,1),kernel_regularizer=regularizers.l2(1e-5)) (main_path3)
#     main_path4 = Concatenate()([main_path4,attention_path4])
#     main_path4 = res_block_decoder(main_path4, [64, 64], [(1, 1), (1, 1)],(1,1))

#     return main_path4, out_3, out_2, out_1

# def three_times_sample(main_path):
#     hyper1 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same',dilation_rate=(1,1),kernel_regularizer=regularizers.l2(1e-5)) (main_path)
#     hyper2 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same',dilation_rate=(1,1), kernel_regularizer=regularizers.l2(1e-5)) (hyper1)
#     hyper3 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same',dilation_rate=(1,1), kernel_regularizer=regularizers.l2(1e-5)) (hyper2)
#     return hyper3


# def two_times_sample(main_path):
#     hyper1 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same',dilation_rate=(1,1), kernel_regularizer=regularizers.l2(1e-5))(main_path)
#     hyper2 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same',dilation_rate=(1,1) ,kernel_regularizer=regularizers.l2(1e-5))(hyper1)
#     return hyper2


# def one_time_sample(main_path):
#     hyper1 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same',dilation_rate=(1,1), kernel_regularizer=regularizers.l2(1e-5)) (main_path)
#     return hyper1


# def join_all_the_layers(hc1,hc2,hc3,hc4):

#     combine1      =  Concatenate()([hc1,hc2])
#     combine2      =  Concatenate()([hc3,hc4])
#     combine_final =  Concatenate()([combine1,combine2])

#     return combine_final

# def aru_gd(input_shape):
#     inputs = Input(shape=input_shape)

#     to_decoder = encoder(inputs)

#     path = res_block(to_decoder[3], [256, 256], [(2, 2), (1, 1)],(8,8))

#     final_out, out_3, out_2, out_1 = decoder(path, from_encoder=to_decoder)

#     final_out1 = Conv2D(filters=1, kernel_size=(1, 1), activation='sigmoid', name='final_output1')(final_out)
#     final_out2= Conv2D(filters=1, kernel_size=(1, 1), activation='sigmoid', name='final_output2')(final_out)
#     # return Model(inputs=inputs, outputs= [final_out, out_3, out_2, out_1])
#     return Model(inputs=inputs, outputs= [final_out1, final_out2])

# from keras import regularizers
# input_shape = (512,512,3)
# model = aru_gd(input_shape)


In [8]:
# # model.compile(optimizer=tf.keras.optimizers.Adam(),
# #               loss={'conv2d_9': bcc_Jaccard_coef_loss, 'conv2d_10': bcc_Jaccard_coef_loss},
# #               metrics={'conv2d_9': [dice_coef, iou, sensitivity, specificity],
# #                        'conv2d_10': [dice_coef, iou, sensitivity, specificity]})
# model.compile(optimizer=tf.keras.optimizers.Adam(),
#               loss={'final_output1': bcc_Jaccard_coef_loss, 'final_output2': bcc_Jaccard_coef_loss},
#               metrics={'final_output1': [dice_coef, iou, sensitivity, specificity],
#                        'final_output2': [dice_coef, iou, sensitivity, specificity]})


# model.summary()

**UNet**

In [9]:
# def conv2d_block(input_tensor, n_filters, kernel_size=3, batchnorm=True):
#     # first layer
#     x = Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size), kernel_initializer="he_normal",
#                padding="same")(input_tensor)
#     if batchnorm:
#         x = BatchNormalization()(x)
#     x = Activation("relu")(x)
#     # second layer
#     x = Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size), kernel_initializer="he_normal",
#                padding="same")(x)
#     if batchnorm:
#         x = BatchNormalization()(x)
#     x = Activation("relu")(x)
#     return x


In [10]:
# def get_unet(input_img, n_filters=16, dropout=0.5, batchnorm=True):
#     # contracting path
#     c1 = conv2d_block(input_img, n_filters=n_filters*1, kernel_size=3, batchnorm=batchnorm)
#     p1 = MaxPooling2D((2, 2)) (c1)
#     p1 = Dropout(dropout*0.5)(p1)

#     c2 = conv2d_block(p1, n_filters=n_filters*2, kernel_size=3, batchnorm=batchnorm)
#     p2 = MaxPooling2D((2, 2)) (c2)
#     p2 = Dropout(dropout)(p2)

#     c3 = conv2d_block(p2, n_filters=n_filters*4, kernel_size=3, batchnorm=batchnorm)
#     p3 = MaxPooling2D((2, 2)) (c3)
#     p3 = Dropout(dropout)(p3)

#     c4 = conv2d_block(p3, n_filters=n_filters*8, kernel_size=3, batchnorm=batchnorm)
#     p4 = MaxPooling2D(pool_size=(2, 2)) (c4)
#     p4 = Dropout(dropout)(p4)

#     c5 = conv2d_block(p4, n_filters=n_filters*16, kernel_size=3, batchnorm=batchnorm)
#     # expansive path
#     u6 = Conv2DTranspose(n_filters*8, (3, 3), strides=(2, 2), padding='same') (c5)
#     u6 = concatenate([u6, c4])
#     u6 = Dropout(dropout)(u6)
#     c6 = conv2d_block(u6, n_filters=n_filters*8, kernel_size=3, batchnorm=batchnorm)

#     u7 = Conv2DTranspose(n_filters*4, (3, 3), strides=(2, 2), padding='same') (c6)
#     u7 = concatenate([u7, c3])
#     u7 = Dropout(dropout)(u7)
#     c7 = conv2d_block(u7, n_filters=n_filters*4, kernel_size=3, batchnorm=batchnorm)

#     u8 = Conv2DTranspose(n_filters*2, (3, 3), strides=(2, 2), padding='same') (c7)
#     u8 = concatenate([u8, c2])
#     u8 = Dropout(dropout)(u8)
#     c8 = conv2d_block(u8, n_filters=n_filters*2, kernel_size=3, batchnorm=batchnorm)

#     u9 = Conv2DTranspose(n_filters*1, (3, 3), strides=(2, 2), padding='same') (c8)
#     u9 = concatenate([u9, c1], axis=3)
#     u9 = Dropout(dropout)(u9)
#     c9 = conv2d_block(u9, n_filters=n_filters*1, kernel_size=3, batchnorm=batchnorm)

#     #outputs = Conv2D(1, (1, 1), activation='sigmoid') (c9)
#     output_BV = layers.Conv2D(1, kernel_size=(1, 1), activation='sigmoid')(c9)
#     output_OD = layers.Conv2D(1, kernel_size=(1, 1), activation='sigmoid')(c9)
#     model = Model(inputs=[input_img], outputs=[output_BV,output_OD])
#     return model
# # Define the input shape for binary semantic segmentation
# input_shape = (512,512,3)  # Example input shape for RGB images

# # Create the TransFuse Encoder-Decoder model with 3 MaxPooling layers for binary semantic segmentation
# #model = get_unet(input_shape)


In [11]:
# input_img = Input((input_shape), name='img')
# model = get_unet(input_img, n_filters=16, dropout=0.05, batchnorm=True)
# model.summary()

In [12]:
# # Load the pre-trained weights into the model
# model.load_weights('/content/drive/MyDrive/Colab_Folder/Model Weights/DRIVE/checkpoints/DRIVEattentionMTL512ep300bs4.h5')

# #

In [13]:
#train_data ='/content/drive/MyDrive/Colab_Notebooks/DRIVE_Augmented/DRIVE_BioNet-Augmented/train' #data path
valid_data = '/content/drive/MyDrive/Colab_Notebooks/DRIVE_Augmented/DRIVE_BioNet-Augmented/test'

In [14]:

import os
import numpy as np
im_height=512
im_width=512
# Get and resize train images and masks
def get_data(path):
    images_paths = os.path.join(path,'images')
    masks_path_BV = os.path.join(path,'GT_BV')
    masks_path_OD = os.path.join(path,'GT_OD')

    # images_ids = os.listdir(images_paths)
    # masks_ids = os.listdir(masks_paths)
    images_ids = sorted(os.listdir(images_paths))
    mask1_ids = sorted(os.listdir(masks_path_BV))
    mask2_ids = sorted(os.listdir(masks_path_OD))

    X = np.zeros((len(images_ids), im_height, im_width, 3), dtype=np.float32)
    y1 = np.zeros((len(mask1_ids), im_height, im_width, 1), dtype=np.float32)
    y2 = np.zeros((len(mask2_ids), im_height, im_width, 1), dtype=np.float32)
    print('Getting and resizing images ... ')
    for n in range (len(images_ids)):
        # Load images
        img = img_to_array(load_img(os.path.join(images_paths,images_ids[n]), color_mode = "rgb"))
        x_img = resize(img, (im_height, im_width, 3), mode='constant', preserve_range=True)

        # Load masks
        mask_BV = img_to_array(load_img(os.path.join(masks_path_BV,mask1_ids[n]), color_mode = "grayscale"))
        mask_BV = resize( mask_BV, (im_height, im_width, 1), mode='constant', preserve_range=True)

        mask_OD = img_to_array(load_img(os.path.join(masks_path_OD, mask2_ids[n]), color_mode = "grayscale"))
        mask_OD = resize(mask_OD, (im_height, im_width, 1), mode='constant', preserve_range=True)
        # Save images
        X[n] = x_img/ 255
        y1[n] = mask_BV/ 255
        y2[n] = mask_OD/255

    print('Done!')
    return X, [y1,y2]


In [15]:
# valid_data = '/content/ISIC2018_512x512/test' #data path
X_test, y_test = get_data(valid_data)


Getting and resizing images ... 
Done!


In [16]:
def evaluate_metrics(y_test, y_pred):
  n = y_pred.shape[0]
  all_F1_score = np.zeros(n)
  all_dice = np.zeros(n)
  all_jaccard = np.zeros(n)
  all_sensitivity = np.zeros(n)
  all_specificity = np.zeros(n)
  for i in range(n):
      gt, pred = y_test[i], y_pred[i]
      gt_flt = np.ndarray.flatten(gt)
      pred_flt = np.ndarray.flatten(pred)

      precisions, recalls, thresholds = precision_recall_curve(gt_flt, pred_flt)
      f1 = 2*(precisions * recalls) / (precisions + recalls)
      max_value = np.argmax(f1)
      thres = thresholds[max_value]
      pred_mask = (pred_flt >= thres)
      pred_label = pred_mask*1

      tn, fp, fn, tp = confusion_matrix(gt_flt, pred_label).ravel()

      F1_score = tp/(tp+((0.5)*(fp+fn)))
      iou = tp / (tp + fp + fn)
      dice = 2*tp / (2*tp + fp + fn)
      specificity = tn / (tn + fp)
      recall = tp / (tp + fn)

      all_F1_score[i] = F1_score
      all_dice[i] = dice
      all_jaccard[i] = iou
      all_sensitivity[i] = recall
      all_specificity[i] = specificity

  print(' F1_score: {:4f}, Dice: {:4f}, Jaccard: {:4f}, Sensitivity: {:4f}, Specificity: {:4f}'.format(
        np.nanmean(all_F1_score), np.nanmean(all_dice), np.nanmean(all_jaccard), np.nanmean(all_sensitivity), np.nanmean(all_specificity)
  ))
  return all_F1_score, all_dice, all_jaccard, all_sensitivity, all_specificity


In [17]:
custom_objects = {"bcc_Jaccard_coef_loss": bcc_Jaccard_coef_loss,"dice_coef":dice_coef,"iou": iou,"sensitivity":sensitivity,"specificity":specificity}

In [21]:
model.load_weights('/content/drive/MyDrive/Colab_Folder/Model Weights/DRIVE/FINAL/Models/ModelTransCNN_300_16.h5')

ValueError: ignored

In [ ]:
# model_path = '/content/drive/MyDrive/Colab_Folder/Model Weights/DRIVE/FINAL/POS_MTLTransCNNep300bs16.h5'
# model = tf.keras.models.load_model(model_path,custom_objects=custom_objects)

#model1 = model.load('/content/drive/MyDrive/Colab_Folder/Model Weights/IOSTAR/FINAL//content/drive/MyDrive/Colab_Folder/Model Weights/DRIVE/FINAL/modelattentionUnetoriginal300_4.h5')
# model2 = tf.keras.models.load_model('path_to_model2.h5')
# model3 = tf.keras.models.load_model('path_to_model3.h5')
# model4 = tf.keras.models.load_model('path_to_model4.h5')
import timeit

# Assuming you have some input_data ready
#input_data = ...  # This could be something like tf.random.normal([batch_size, input_shape])
test_image = load_img('/content/drive/MyDrive/Colab_Notebooks/DRIVE_Augmented/DRIVE_BioNet-Augmented/test/images/15.png', target_size=(512, 512))
test_image = img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)
print(test_image.shape)
# Scale the pixel values to [0, 1] range
test_image = test_image / 255.
def predict_model1():
   predict_model1= model.predict(test_image, verbose=1)

# def predict_model2():
#     model2.predict(input_data)

# def predict_model3():
#     model3.predict(input_data)

# def predict_model4():
#     model4.predict(input_data)

time_model1 = timeit.timeit(predict_model1, number=5)  # Adjust the number as needed
# time_model2 = timeit.timeit(predict_model2, number=10)  # Adjust the number as needed
# time_model3 = timeit.timeit(predict_model3, number=10)  # Adjust the number as needed
# time_model4 = timeit.timeit(predict_model4, number=10)  # Adjust the number as needed

print(f"Model 1 Prediction Time: {time_model1:.5f} seconds")
# print(f"Model 2 Prediction Time: {time_model2:.5f} seconds")
# print(f"Model 3 Prediction Time: {time_model3:.5f} seconds")
# print(f"Model 4 Prediction Time: {time_model4:.5f} seconds")


In [ ]:
per_model = time_model1/5
print(per_model)


In [ ]:
# Convert ground truth labels to binary
y_test_BV = (y_test[0] > 0.5).astype(np.uint8)
y_test_OD = (y_test[1] > 0.5).astype(np.uint8)

start_time = time.time()
predictions = model.predict(X_test, batch_size=1, verbose=1)

end_time = time.time()
inference_time = end_time - start_time
print("Inference Time: {:.2f} seconds".format(inference_time))
inference_time_per_image = inference_time/len(X_test)
print("Inference Time per image: {:.2f} seconds".format(inference_time_per_image ))


In [ ]:
# Convert model's outputs to binary
predictions_BV = (predictions[0] > 0.5).astype(np.uint8)
predictions_OD = (predictions[1] > 0.5).astype(np.uint8)

# Evaluate metrics for each task
eval_BV = evaluate_metrics(y_test_BV, predictions_BV)
eval_OD = evaluate_metrics(y_test_OD, predictions_OD)

In [ ]:
memory_usage = model.count_params() * 4  # Assuming float32 data type (4 bytes per parameter)
print("Memory Usage: {:.2f} MB".format(memory_usage / 1024 / 1024))  # Divide by 1024 twice to convert from bytes to KB, then KB to MB


In [ ]:
test_image = load_img('/content/drive/MyDrive/Colab_Notebooks/DRIVE_Augmented/DRIVE_BioNet-Augmented/test/images/15.png', target_size=(512, 512))
test_image = img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)

# Scale the pixel values to [0, 1] range
test_image = test_image / 255.

# Use the model to predict the mask for the test image

start_time = time.time()
preds_test = model.predict(test_image)
end_time = time.time()
inference_time = end_time - start_time
print("Inference Time: {:.2f} seconds".format(inference_time))


In [ ]:
import numpy as np

fig, ax = plt.subplots(nrows=1, ncols=3)
ax[0].imshow(test_image[0])
ax[0].axis('off')
ax[1].imshow(np.squeeze(preds_test[0]), cmap='gray') # squeeze out unnecessary dimensions
ax[1].axis('off')
ax[2].imshow(np.squeeze(preds_test[1]), cmap='gray')
ax[2].axis('off')

#fig, ax = plt.subplots(nrows=1, ncols=2)
#ax[0].imshow(test_image[0])
#ax[1].imshow(preds_test[0], cmap='gray') # you may need

In [ ]:
  !pip install keras-flops
  from keras_flops import get_flops

  flops = get_flops(model)
  print(f"FLOPs: {flops / 10**9:.03} G")

In [ ]:
!cat /proc/cpuinfo

In [ ]:
!nvidia-smi


In [ ]:
!cat /proc/meminfo
with open("/proc/meminfo", "r") as f:
    lines = f.readlines()

for line in lines:
    if "MemTotal" in line:
        print(line.strip())
        break

